## 使用 History

illufly 支持基于文件或 redis 等数据库持久化。

### 空的历史

In [4]:
from illufly.chat import ChatQwen

chat = ChatQwen(name="qwen")
chat.thread_ids

[]

In [5]:
chat.load_memory(-1)
chat.memory

[]

In [6]:
chat.thread_id

-1

## 写入历史对话

In [1]:
from illufly.chat import ChatQwen

chat = ChatQwen(name="qwen")
chat("给我写一首2句儿歌")

[USER] 给我写一首2句儿歌
小星星，亮晶晶，  
天上地下来放灯。


'小星星，亮晶晶，  \n天上地下来放灯。'

In [2]:
chat = ChatQwen(name="qwen")
chat("给我写一首2句儿歌")

[USER] 给我写一首2句儿歌
小兔跳跳真可爱，太阳公公笑开怀。


'小兔跳跳真可爱，太阳公公笑开怀。'

## 加载历史对话

In [1]:
from illufly.chat import ChatQwen

chat = ChatQwen(name="qwen")
chat.load_memory(-2)
chat.memory

[{'role': 'user', 'content': '给我写一首2句儿歌'},
 {'role': 'assistant', 'content': '小星星，亮晶晶，  \n天上地下来放灯。'}]

In [2]:
chat("星星能改成月亮？")
chat.memory

[USER] 星星能改成月亮？
当然可以，改后的儿歌如下：

小月亮，亮晶晶，  
天上地下来放灯。


'当然可以，改后的儿歌如下：\n\n小月亮，亮晶晶，  \n天上地下来放灯。'